## Test window-method declustering

In [1]:
import eqcatalog

In [2]:
def render_html(html):
    from IPython.core.display import display, HTML
    display(HTML(html))

In [31]:
reload(eqcatalog)

<module 'eqcatalog' from 'E:\Home\_kris\Python\seismo\eqcatalog\__init__.pyc'>

In [3]:
cat = eqcatalog.rob.seismodb.query_local_eq_catalog(start_date=2018, end_date=2018)
cat.print_info()

Name: ROB Catalog 2018-01-01 - 2018-12-31
Earthquake number: 66
Start time: 2018-01-01T00:00:00.000
End time :  2018-12-31T00:00:00.000
ML: n=66, min=-0.4, max=3.6
Longitude bounds: -0.3414 / 7.9002
Latitude bounds: 50.1087 / 51.4283
Depth range: 0.0 - 21.5 km


In [32]:
fa_ratio = 0.
dc_method = eqcatalog.declustering.WindowMethod(fa_ratio)
#dc_window_name = 'Uhrhammer1986'
#dc_window_name = 'GardnerKnopoff1974'
dc_window_name = "Gruenthal2009"
dc_window = eqcatalog.declustering.get_window_by_name(dc_window_name)
Mrelation = "default"

In [33]:
dc_cat1 = cat.subselect_declustering(dc_method, dc_window, Mrelation)
dc_cat2 = cat.subselect_declustering_legacy("Window", dc_window_name, fa_ratio, Mtype='MW', Mrelation=Mrelation)
print(len(dc_cat1), len(dc_cat2), dc_cat1 == dc_cat2)

True
True
True
True
True
True
True
True
True
True
(39, 39, False)


In [24]:
diff_cat = dc_cat1.get_difference(dc_cat2)
render_html(diff_cat.print_list(as_html=True))

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6505,2018-04-21,11:47:47,DUEREN (DE),6.5312,50.8597,16.5,2.7,nan,nan
6625,2018-05-25,22:43:27,KINROOI (BE),5.6872,51.1750,16.6,3.1,nan,nan
6965,2018-07-07,13:10:51,ALSDORF (DE),6.1883,50.8743,12.8,0.9,nan,nan


In [25]:
render_html(dc_cat2.get_difference(dc_cat1).print_list(as_html=True))

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6525,2018-04-18,09:17:59,GOLZHEIM (DE),6.5895,50.8283,13.8,0.9,nan,nan
6635,2018-05-25,22:42:12,KINROOI (BE),5.6800,51.1000,15.0,1.0,nan,nan
6960,2018-07-06,21:50:50,ALSDORF (DE),6.1818,50.8733,12.5,0.7,nan,nan


In [26]:
dc_result = dc_method.analyze_clusters(cat, dc_window, Mrelation, verbose=False)
dc_cat1b = dc_result.get_declustered_catalog()
print(len(dc_cat1), dc_result.get_num_independent_events(),  dc_cat1b== dc_cat1)
dc_result.print_info()

(39, 38, False)
Number of clusters identified: 8
Max. cluster length: 14
Num. clustered/unclustered events: 36 / 30
Num. dependent/independent events: 28 / 38


In [27]:
render_html(dc_cat1.get_difference(dc_cat1b).print_list(as_html=True))

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6965,2018-07-07,13:10:51,ALSDORF (DE),6.1883,50.8743,12.8,0.9,nan,nan


In [28]:
render_html(dc_cat1b.get_difference(dc_cat1).print_list(as_html=True))

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW


In [29]:
for cluster in dc_result.get_clusters():
    render_html(cluster.to_catalog().print_list(as_html=True))
    print('')

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6525,2018-04-18,09:17:59,GOLZHEIM (DE),6.5895,50.8283,13.8,0.9,nan,nan
6505,2018-04-21,11:47:47,DUEREN (DE),6.5312,50.8597,16.5,2.7,nan,nan
6520,2018-04-23,23:52:04,HURTGENWALD (DE),6.3540,50.6750,7.1,-0.1,nan,nan
6960,2018-07-06,21:50:50,ALSDORF (DE),6.1818,50.8733,12.5,0.7,nan,nan
6965,2018-07-07,13:10:51,ALSDORF (DE),6.1883,50.8743,12.8,0.9,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
7290,2018-11-17,05:25:01,RAFFELSBRAND (DE),6.2972,50.6540,5.8,2.6,nan,nan
7320,2018-11-17,05:30:19,LAMMERSDORF (DE),6.3157,50.6848,5.4,0.0,nan,nan
7325,2018-11-17,05:33:40,LAMMERSDORF (DE),6.3120,50.6892,5.1,0.2,nan,nan
7335,2018-11-17,05:36:29,LAMMERSDORF (DE),6.3162,50.6825,5.1,0.0,nan,nan
7345,2018-11-17,05:36:38,LAMMERSDORF (DE),6.3158,50.6918,5.0,0.1,nan,nan
7310,2018-11-17,22:09:46,LAMMERSDORF (DE),6.3028,50.6880,4.2,0.5,nan,nan
7350,2018-11-17,22:10:04,LAMMERSDORF (DE),6.3128,50.6942,5.1,-0.1,nan,nan
7355,2018-11-19,03:40:11,LAMMERSDORF (DE),6.3092,50.6872,4.6,-0.4,nan,nan
7425,2018-11-25,07:45:12,HURTGENWALD (DE),6.3212,50.6878,5.0,0.4,nan,nan
7465,2018-11-28,02:57:40,LAMMERSDORF (DE),6.3025,50.6765,3.3,0.5,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6245,2018-01-22,07:20:35,SAUERTHAL (DE),7.8895,50.1087,16.6,2.4,nan,nan
6250,2018-01-22,07:35:58,SAUERTHAL (DE),7.9002,50.1128,16.7,2.3,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6970,2018-07-18,03:40:50,OCHTENDUNG (DE),7.4050,50.3660,9.9,2.4,nan,nan
7020,2018-07-24,21:53:37,NIEDERDURENBACH (DE),7.1855,50.4540,11.7,1.1,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6995,2018-07-23,08:38:05,HEERLEN (NL),5.9872,50.8963,5.6,2.2,nan,nan
7015,2018-07-23,08:39:03,HEERLEN (NL),5.9920,50.9055,8.4,1.3,nan,nan
7010,2018-07-23,15:01:31,STOLBERG (DE),6.2838,50.7238,4.3,0.6,nan,nan
7005,2018-07-23,22:14:02,LANDGRAAF (NL),5.9942,50.9140,8.6,0.8,nan,nan
7000,2018-07-23,22:14:16,HEERLEN (NL),5.9867,50.9030,8.5,1.6,nan,nan
7045,2018-07-30,01:13:50,VOERENDAAL (NL),5.9042,50.8648,1.4,0.5,nan,nan
7050,2018-07-30,01:13:51,LANDGRAAF (NL),6.0025,50.9100,4.2,1.5,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6585,2018-05-13,12:06:58,ROTH BEI PRUM (DE),6.3757,50.3018,21.5,1.3,nan,nan
6590,2018-05-13,15:23:46,AUW BEI PRUM (DE),6.4097,50.3142,21.0,1.0,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
7535,2018-12-27,11:38:08,SIEVERNICH (DE),6.7027,50.7422,16.3,1.1,nan,nan
7555,2018-12-29,18:37:40,SIEVERNICH (DE),6.7347,50.7327,14.3,0.9,nan,nan


ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6635,2018-05-25,22:42:12,KINROOI (BE),5.6800,51.1000,15.0,1.0,nan,nan
6625,2018-05-25,22:43:27,KINROOI (BE),5.6872,51.1750,16.6,3.1,nan,nan


In [12]:
time_deltas = diff_cat.get_inter_event_times('h')
mags = diff_cat.get_magnitudes('MW', Mrelation)
distances = [diff_cat[0].hypocentral_distance(diff_cat[1]), diff_cat[1].hypocentral_distance(diff_cat[2])]
for td_obs, mag, dist in zip(time_deltas, mags, distances):
    max_td = eqcatalog.time_functions.fractional_time_delta(dc_window.get_time_window(mag), 'h')
    max_dist = dc_window.get_dist_window(mag)
    print(mag, td_obs, max_td, td_obs < max_td, dist, max_dist, dist < max_dist)

(3.143333333333331, 1022.4566666666667, 341.5388888888889, False, 48.586994264277656, 23.558338987013745, False)
(1.5299999999999976, 540.05, 45.79194444444445, False, 16.110565027802622, 14.873528158636354, False)


In [13]:
for eq in diff_cat[:2]:
    print(dc_method.is_in_dist_window(eq, eq.get_MW(Mrelation), diff_cat, dc_window))

[ True False False]
[False  True False]


In [14]:
eq = cat.get_event_by_id(6995)
eq = cat[29]
print(dc_method.is_in_window(eq, eq.get_MW(Mrelation), diff_cat, dc_window))

[False False False]


In [30]:
cat.index(str(6965))

26

In [16]:
render_html(cat.print_list(as_html=True))

ID,Date,Time,Name,Lon,Lat,Depth,ML,MS,MW
6225,2018-01-17,02:15:29,VALKENBURG AAN DE GEUL (NL),5.8203,50.8345,16.5,0.7,nan,nan
6245,2018-01-22,07:20:35,SAUERTHAL (DE),7.8895,50.1087,16.6,2.4,nan,nan
6250,2018-01-22,07:35:58,SAUERTHAL (DE),7.9002,50.1128,16.7,2.3,nan,nan
6275,2018-01-28,22:33:07,HURTGENWALD (DE),6.2967,50.7030,4.1,0.6,nan,nan
6280,2018-02-01,23:14:37,RURBERG (DE),6.3793,50.6625,4.9,-0.2,nan,nan
6300,2018-02-03,03:50:32,PLAIDT (DE),7.3948,50.3200,17.7,1.4,nan,nan
6285,2018-02-07,12:58:54,NEDERWEERT (NL),5.7567,51.2400,14.9,1.7,nan,nan
6355,2018-02-23,23:07:00,SAINT-VAAST (BE),4.1552,50.4532,2.5,0.7,nan,nan
6365,2018-02-26,02:05:05,LIPPORN (DE),7.8580,50.1403,10.2,1.6,nan,nan
6395,2018-03-11,19:31:35,MARIAWEILER (DE),6.4630,50.8210,16.2,0.6,nan,nan
